In [291]:
import pandas as pd 
from pulp import *
import re
import numpy as np

# Data Preparation

In [493]:
with open('data_1.txt', 'r') as fp:
    x1 = fp.readlines()

with open('data_2.txt', 'r') as fp:
    x2 = fp.readlines()

x1 = [re.sub('\n','',i) for i in x1]
x2 = [re.sub('\n','',i) for i in x2]

In [494]:
df = pd.DataFrame([tuple(re.split(',',i)) for i in x1])
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df.reset_index(drop=True, inplace=True)

df_2 = pd.DataFrame([tuple(re.split(',',i)) for i in x2])
df_2.columns = df_2.iloc[0]
df_2.drop(df_2.index[0], inplace=True)
df_2.reset_index(drop=True, inplace=True)

In [495]:
df[' Inventory Cost(100 units)']=df[' Inventory Cost(100 units)'].astype(int)

In [496]:
df

,Plant,Inventory Cost(100 units),Vendor,Capacity,Disruption Probability(1 - µ),TTR of vendor
0,ABX,31,1,1.2,0.0667,1
1,ABN,30,1,1.2,0.0667,1
2,GHY,32,2,0.9,0.1333,1.2
3,KJU,29,3,0.5,0.2000,1.4
4,PFR,30,4,0.5,0.1667,1.6
5,JKI1,30,5,2.4,0.0667,1.8
6,JKI2,33,5,2.4,0.0667,1.8
7,JKI3,31,5,2.4,0.0667,1.8
8,CFG1,500,6,0.2,0.0667,2
9,CFG2,550,7,0.4,0.1667,2.2


In [497]:
df_2

,ItemID,Vendor,Geographic Location,CFG1,CFG2,CFG3,CFG4
0,ABX,1,US,1,1,1,0
1,ABN,1,US,1,1,1,0
2,GHY,2,Mexico,1,1,2,1
3,KJU,3,Mexico,1,0,1,0
4,PFR,4,US,1,0,1,0
5,JKI1,5,Mexico,0,3,0,0
6,JKI2,5,Mexico,3,0,0,0
7,JKI3,5,Mexico,2,0,3,2
8,CFG1,6,US,1,0,0,0
9,CFG2,7,US,0,1,0,0


In [513]:
d = {}
for i in range(8):
    item = df_2['ItemID'].iloc[i]
    cfg1 = df_2[' CFG1'].iloc[i] 
    cfg2 = df_2[' CFG2'].iloc[i]
    cfg3 = df_2[' CFG3'].iloc[i]
    cfg4 = df_2[' CFG4'].iloc[i]
    d[(item,'CFG1')] = int(cfg1)
    d[(item,'CFG2')] = int(cfg2)
    d[(item,'CFG3')] = int(cfg3)
    d[(item,'CFG4')] = int(cfg4)
B = d

# Defining Parameters

In [514]:
# Create a list of the Plant Nodes
M = df['Plant'].tolist()
M1 = df['Plant'].tolist()[0:8]
M2 = df['Plant'].tolist()[8:]

# Create a dictionary of capacity for all Plant nodes
capacity = dict(zip(M1,df[' Capacity']))

# Create a dictionary of disruption probability for all Plant nodes
dis_prob = dict(zip(M2,df[' Disruption Probability(1 - µ)']))

# Create a dictionary of inventory costs
inv_cost = dict(zip(M1,df[' Inventory Cost(100 units)']))


# Demand for different products
demand = {'CFG1':27, 'CFG2':107, 'CFG3':203, 'CFG4':40}

# Lost Sales Penalty
f = {'CFG1':2.7, 'CFG2':1.7, 'CFG3':2.3, 'CFG4':1.5}

# Defining Random Variables

In [515]:
np.random.seed(9)
v = np.random.randint(0,2,10)
v

array([0, 0, 0, 1, 0, 0, 1, 0, 1, 1])

In [516]:
TR = -1

# Problem Definition

In [587]:
obj = LpProblem("Disruption_Risk_Mitigation",LpMinimize)

# Defining Decision Variables

In [588]:
# Since x is Continuous
x  = {(i,j):
           LpVariable(cat=LpContinuous, 
               lowBound=0, upBound=100, 
               name="x_{0}_{1}".format(i,j)) 
for i in M1 for j in M2}


In [589]:
# This variable indicates lost sales
l = LpVariable.dict("lost_sales", M2, lowBound=0,cat='Continuous')

In [590]:
u = LpVariable.dicts("good_prod", M1, lowBound=0,cat=LpInteger)

In [591]:
r = LpVariable.dicts("inventory_prod", M1, lowBound=0,cat=LpContinuous)

# Defining Objective

In [592]:
obj += lpSum([f[i]*l[i] for i in M2] + [inv_cost[i]*r[i]*0.01 for i in M1])

# Defining Constraints

In [595]:
# # Constraint #0
# for i in M1:
#     obj += lpSum(x[(i,j)] for j in M2) <= 3

In [596]:
# Constraint #1
for j in M2:
    obj += lpSum(x[(i,j)] for i in M1) + l[j] >= demand[j]

In [597]:
# Constraint #2
for i in M1:
    obj += lpSum(x[(i,j)] for j in M2) + u[i] <= r[i]

In [598]:
# # Constraint #3
# for j in M2:
#      obj += lpSum([x[(i,j)] for i in M1])/lpSum([B[(i,j)] for i in M1]) >= u[i]

In [599]:
obj.solve()

1

In [600]:
for v in obj.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

inventory_prod_ABN = 3.0
inventory_prod_ABX = 3.0
inventory_prod_GHY = 3.0
inventory_prod_JKI1 = 3.0
inventory_prod_JKI2 = 3.0
inventory_prod_JKI3 = 3.0
inventory_prod_KJU = 3.0
inventory_prod_PFR = 3.0
lost_sales_CFG1 = 3.0
lost_sales_CFG2 = 107.0
lost_sales_CFG3 = 203.0
lost_sales_CFG4 = 40.0
x_ABN_CFG1 = 3.0
x_ABX_CFG1 = 3.0
x_GHY_CFG1 = 3.0
x_JKI1_CFG1 = 3.0
x_JKI2_CFG1 = 3.0
x_JKI3_CFG1 = 3.0
x_KJU_CFG1 = 3.0
x_PFR_CFG1 = 3.0


In [601]:
value(obj.objective)

724.2799999999999